In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import metrics


BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.metrics

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
EVAL_GROUPS = CONFIG['experiment_configs'][DATASET]['eval_groups']
RANDOM_SEED = CONFIG['random_seed']
EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]

print(RANDOM_SEED)

PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')

BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base") # mt = model_type

if os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has been done for rs={RANDOM_SEED}")

15


In [4]:
train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "train.csv"))
hyper_train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_train.csv"))
val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "val.csv"))
hyper_val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_val.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DIR, "test.csv"))

val_full_df = pd.concat([val_df, hyper_val_df])

In [5]:
x_train = train_df.drop([*EVAL_GROUPS, 'label'], axis=1).values
y_train = train_df['label'].values

x_hyper_train = hyper_train_df.drop([*EVAL_GROUPS, 'label'], axis=1).values
y_hyper_train = hyper_train_df['label'].values

x_val_full = val_full_df.drop([*EVAL_GROUPS, 'label'], axis=1).values
y_val_full = val_full_df['label'].values

x_test = test_df.drop([*EVAL_GROUPS, 'label'], axis=1).values
y_test = test_df['label'].values

In [6]:
y_train = tf.keras.utils.to_categorical(y_train)
y_hyper_train = tf.keras.utils.to_categorical(y_hyper_train)
y_val_full = tf.keras.utils.to_categorical(y_val_full)
y_test = tf.keras.utils.to_categorical(y_test)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])

In [8]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Need to save the best model by validation loss
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=BASE_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

callbacks = [save_best]

In [10]:
model.fit(
    x_train,
    y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (x_hyper_train, y_hyper_train),
    callbacks=callbacks,
)

Epoch 1/100
679/682 [============================>.] - ETA: 0s - loss: 0.4755 - accuracy: 0.7782
Epoch 00001: val_loss improved from inf to 0.41096, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 3ms/step - loss: 0.4751 - accuracy: 0.7785 - val_loss: 0.4110 - val_accuracy: 0.8048
Epoch 2/100
657/682 [===========================>..] - ETA: 0s - loss: 0.3922 - accuracy: 0.8185
Epoch 00002: val_loss improved from 0.41096 to 0.37396, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3908 - accuracy: 0.8195 - val_loss: 0.3740 - val_accuracy: 0.8264
Epoch 3/100
682/682 [==============================] - ETA: 0s - loss: 0.3670 - accuracy: 0.8318
Epoch 00003: val_loss improved from 0.37396 to 0.35864, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3670 - accuracy: 

Epoch 25/100
673/682 [============================>.] - ETA: 0s - loss: 0.3179 - accuracy: 0.8530
Epoch 00025: val_loss improved from 0.31901 to 0.31887, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3174 - accuracy: 0.8536 - val_loss: 0.3189 - val_accuracy: 0.8526
Epoch 26/100
665/682 [============================>.] - ETA: 0s - loss: 0.3163 - accuracy: 0.8536
Epoch 00026: val_loss improved from 0.31887 to 0.31831, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3170 - accuracy: 0.8532 - val_loss: 0.3183 - val_accuracy: 0.8530
Epoch 27/100
657/682 [===========================>..] - ETA: 0s - loss: 0.3171 - accuracy: 0.8533
Epoch 00027: val_loss improved from 0.31831 to 0.31805, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3167 - acc

675/682 [============================>.] - ETA: 0s - loss: 0.3115 - accuracy: 0.8565
Epoch 00049: val_loss improved from 0.31510 to 0.31508, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3124 - accuracy: 0.8559 - val_loss: 0.3151 - val_accuracy: 0.8549
Epoch 50/100
670/682 [============================>.] - ETA: 0s - loss: 0.3121 - accuracy: 0.8559
Epoch 00050: val_loss improved from 0.31508 to 0.31500, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3122 - accuracy: 0.8559 - val_loss: 0.3150 - val_accuracy: 0.8544
Epoch 51/100
678/682 [============================>.] - ETA: 0s - loss: 0.3121 - accuracy: 0.8558
Epoch 00051: val_loss improved from 0.31500 to 0.31476, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3122 - accuracy: 0.8557

Epoch 74/100
666/682 [============================>.] - ETA: 0s - loss: 0.3098 - accuracy: 0.8578
Epoch 00074: val_loss did not improve from 0.31402
682/682 [==============================] - 2s 2ms/step - loss: 0.3107 - accuracy: 0.8573 - val_loss: 0.3141 - val_accuracy: 0.8555
Epoch 75/100
681/682 [============================>.] - ETA: 0s - loss: 0.3108 - accuracy: 0.8567
Epoch 00075: val_loss improved from 0.31402 to 0.31392, saving model to ../../../models/adult_bb/rs=15/adult_bb_mt=base.h5
682/682 [==============================] - 2s 2ms/step - loss: 0.3108 - accuracy: 0.8568 - val_loss: 0.3139 - val_accuracy: 0.8551
Epoch 76/100
658/682 [===========================>..] - ETA: 0s - loss: 0.3091 - accuracy: 0.8576
Epoch 00076: val_loss did not improve from 0.31392
682/682 [==============================] - 2s 2ms/step - loss: 0.3106 - accuracy: 0.8567 - val_loss: 0.3140 - val_accuracy: 0.8556
Epoch 77/100
679/682 [============================>.] - ETA: 0s - loss: 0.3109 - accurac

679/682 [============================>.] - ETA: 0s - loss: 0.3102 - accuracy: 0.8572
Epoch 00100: val_loss did not improve from 0.31369
682/682 [==============================] - 2s 2ms/step - loss: 0.3100 - accuracy: 0.8573 - val_loss: 0.3137 - val_accuracy: 0.8559
